In [1]:
from google.cloud import bigquery

In [2]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/pavannaik/Desktop/API/sql-kaggle-244915-e2d3c0bfe87b.json"

In [3]:
client = bigquery.Client()

In [4]:
dataset_ref = client.dataset("world_bank_intl_education", project="bigquery-public-data")
dataset = client.get_dataset(dataset_ref)
table_ref = dataset_ref.table("international_education")
table = client.get_table(table_ref)
client.list_rows(table, max_results=5).to_dataframe()

,country_name,country_code,indicator_name,indicator_code,value,year
0,East Asia & Pacific,EAS,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,9.857965e+03,2016
1,Euro area,EMU,"GDP per capita, PPP (constant 2011 internation...",NY.GDP.PCAP.PP.KD,3.833465e+04,2016
2,Europe & Central Asia (excluding high income),ECA,GNI (current US$),NY.GNP.MKTP.CD,2.920391e+12,2016
3,Least developed countries: UN classification,LDC,"Population, ages 0-14, female",SP.POP.0014.FE.IN,1.928885e+08,2016
4,Middle East & North Africa,MEA,"GDP, PPP (current international $)",NY.GDP.MKTP.PP.CD,8.548505e+12,2016


Government expenditure on Education

In [5]:
country_spend_pct_query = """
                          SELECT country_name, AVG(value) as avg_ed_spending_pct
                          FROM `bigquery-public-data.world_bank_intl_education.international_education`
                          WHERE indicator_code = "SE.XPD.TOTL.GD.ZS" AND year>=2010 AND year<= 2017
                          GROUP BY country_name
                          ORDER BY avg_ed_spending_pct DESC
                          """

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
country_spend_pct_query_job = client.query(country_spend_pct_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
country_spending_results = country_spend_pct_query_job.to_dataframe()

# View top few rows of results
print(country_spending_results.head())

            country_name  avg_ed_spending_pct
0                   Cuba            12.837270
1  Micronesia, Fed. Sts.            12.467750
2        Solomon Islands            10.001080
3                Moldova             8.372153
4                Namibia             8.349610


In [6]:

code_count_query = """SELECT indicator_name, indicator_code, COUNT(indicator_code) as num_rows
FROM `bigquery-public-data.world_bank_intl_education.international_education`
WHERE year = 2016
GROUP BY indicator_code, indicator_name
HAVING COUNT(indicator_code) >= 175 
ORDER BY num_rows DESC
"""

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
code_count_query_job = client.query(code_count_query, job_config=safe_config)


code_count_results = code_count_query_job.to_dataframe()


print(code_count_results.head())

                    indicator_name     indicator_code  num_rows
0                Population, total        SP.POP.TOTL       232
1     Population growth (annual %)        SP.POP.GROW       232
2  Internet users (per 100 people)     IT.NET.USER.P2       223
3    Population, ages 0-14, female  SP.POP.0014.FE.IN       213
4     Population, ages 0-14, total     SP.POP.0014.TO       213
